# Underetanding Python decorator

## Function

### Functions as First-Class Objects

In [1]:
def foo(s):
    return s
    
def bar(foo):
    def new_func():
        pass
    new_func()
    return foo

# result = bar(foo("foo"))
# print(result)

result = bar(foo)
print(result("foo"))

foo


### Closures in Python

In [2]:
def bar():
    prefix = "Hi"
    def new_func(s):
        return f"{prefix}: {s}"
    return new_func

result = bar()
print(result("foo"))

del bar
try:
    result = bar(foo)
except NameError:
    print("bar is gone")
print(result("long live foo"))

print(result.__code__.co_freevars)
print(result.__closure__[0].cell_contents)


Hi: foo
bar is gone
Hi: long live foo
('prefix',)
Hi


In [3]:
def bar(grettings):
    def new_func(s):
        return f"{grettings}: {s}"
    return new_func

warpped_func1 = bar("Hi")
warpped_func2 = bar("Hey")
print(warpped_func1("foo"))
print(warpped_func2("bar"))

print(warpped_func1.__code__.co_freevars)
print(warpped_func1.__closure__[0].cell_contents)

print(warpped_func1.__code__.co_freevars)
print(warpped_func1.__closure__[0].cell_contents)

Hi: foo
Hey: bar
('grettings',)
Hi
('grettings',)
Hi


## Decorator

### Basic decorator

In [4]:
def foo(s):
    """this function just pass a string"""
    return s
    
def verbose(func):
    def wrapper(s):
        print(f"Entering {func.__name__}")
        func(s)
        print(f"Exiting {func.__name__}")
    return wrapper

warpped_func1 = verbose(foo)
warpped_func1("foo")

@verbose
def bar(s):
    """this function just pass a string"""
    return s

bar("bar")
print(foo.__doc__)
print(bar.__doc__)

Entering foo
Exiting foo
Entering bar
Exiting bar
this function just pass a string
None


### Update a wrapper function to look like the wrapped function

ref: https://github.com/python/cpython/blob/v3.8.2/Lib/functools.py#L63

In [5]:
from functools import wraps

def verbose(func):
    @wraps(func)
    def wrapper(s):
        print(f"Entering {func.__name__}")
        func(s)
        print(f"Exiting {func.__name__}")
    return wrapper

@verbose
def bar(s):
    """this function just pass a string"""
    return s

print(bar.__doc__)

result = bar("bar")
print(result)

this function just pass a string
Entering bar
Exiting bar
None


### Return

In [6]:
from functools import wraps

def verbose(func):
    @wraps(func)
    def wrapper(s):
        print(f"Entering {func.__name__}")
        return func(s)
        print(f"Exiting {func.__name__}")
    return wrapper

@verbose
def bar(s):
    """this function just pass a string"""
    return s

result = bar("bar")
print(result)

Entering bar
bar


In [7]:
from functools import wraps

def verbose(func):
    @wraps(func)
    def wrapper(s):
        print(f"Entering {func.__name__}")
        original_result = func(s)
        print(f"Exiting {func.__name__}")
        return original_result
    return wrapper

@verbose
def bar(s):
    """this function just pass a string"""
    return s

result = bar("bar")
print(result)

Entering bar
Exiting bar
bar


### Parameters

In [8]:
from functools import wraps

def verbose(func):
    @wraps(func)
    def wrapper(s):
        print(f"Entering {func.__name__}")
        original_result = func(s)
        print(f"Exiting {func.__name__}")
        return original_result
    return wrapper

@verbose
def foo(x, y):
    """this function just pass two strings"""
    return x, y

@verbose
def bar(s):
    """this function just pass a string"""
    return s

result1 = bar("bar")
# TypeError: new_func() takes 1 positional argument but 2 were given
# result2 = foo("foo", "bar")

Entering bar
Exiting bar


In [9]:
from functools import wraps

def verbose(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Entering {func.__name__}")
        original_result = func(*args, **kwargs)
        print(f"Exiting {func.__name__}")
        return original_result
    return wrapper

@verbose
def foo(x, y):
    """this function just pass two strings"""
    return x, y

@verbose
def bar(s):
    """this function just pass a string"""
    return s

@verbose
def fool(*args, **kwargs):
    """this function just pass two strings"""
    return f"{args}/{kwargs}"

result1 = bar("bar")
result2 = foo("foo", "bar")
# positional argument follows keyword argument
result3 = fool("foo", "bar", keyword1="xyz", keyword2="abc")
print(result3)

Entering bar
Exiting bar
Entering foo
Exiting foo
Entering fool
Exiting fool
('foo', 'bar')/{'keyword1': 'xyz', 'keyword2': 'abc'}


### Class

In [10]:
class Verbose(object):
    def __init__(self, f):
        self.f = f

    def __call__(self, *args, **kwargs):
        print(f"Entering {self.f.__name__}")
        original_result = self.f(*args, **kwargs)
        print(f"Exiting {self.f.__name__}")
        return original_result

@Verbose
def fool(*args, **kwargs):
    """this function just pass two strings"""
    return f"{args}/{kwargs}"

result3 = fool("foo", "bar", keyword1="xyz", keyword2="abc")
print(result3)

print(f"{callable(Verbose)}/{callable('s')}")

Entering fool
Exiting fool
('foo', 'bar')/{'keyword1': 'xyz', 'keyword2': 'abc'}
True/False


In [11]:
def verbose(wrapped):
    return Verbose(wrapped)

class Verbose():
    def __init__(self, f):
        self.f = f

    def __call__(self, *args, **kwargs):
        print(f"Entering {self.f.__name__}")
        original_result = self.f(*args, **kwargs)
        print(f"Exiting {self.f.__name__}")
        return original_result

@verbose
def fool(*args, **kwargs):
    """this function just pass two strings"""
    return f"{args}/{kwargs}"

result3 = fool("foo", "bar", keyword1="xyz", keyword2="abc")
print(result3)

print(f"{callable(Verbose)}/{callable('s')}")

Entering fool
Exiting fool
('foo', 'bar')/{'keyword1': 'xyz', 'keyword2': 'abc'}
True/False


In [12]:
import logging
import sys

logging.basicConfig(level=logging.INFO)

class Class1:
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__qualname__)
        # self.logger.addHandler(logging.StreamHandler(stream=sys.stdout))
        self.logger.info("Init...")
    def method1(self):
        self.logger.info("metod1...")

class Class2:
    def __init__(self):
        self.logger = logging.getLogger(self.__class__.__qualname__)
        # self.logger.addHandler(logging.StreamHandler(stream=sys.stdout))
        self.logger.info("Init...")
    def method1(self):
        self.logger.info("metod1...")

class1 = Class1()
class1.method1()
class2 = Class2()
class2.method1()

INFO:Class1:Init...
INFO:Class1:metod1...
INFO:Class2:Init...
INFO:Class2:metod1...


In [13]:
import logging
import sys

logging.basicConfig(level=logging.INFO)

def logged(class_):
    class_.logger = logging.getLogger(class_.__qualname__)
    # class_.logger.addHandler(logging.StreamHandler(stream=sys.stdout))
    return class_

@logged
class Class1:
    def __init__(self):
        self.logger.info("Init...")
    def method1(self):
        self.logger.info("metod1...")

@logged
class Class2:
    def __init__(self):
        self.logger.info("Init...")
    def method1(self):
        self.logger.info("metod1...")

class1 = Class1()
class1.method1()
class2 = Class2()
class2.method1()

INFO:Class1:Init...
INFO:Class1:metod1...
INFO:Class2:Init...
INFO:Class2:metod1...
